# FAISS Indexes: IVF and HNSW

Learn about approximate indexes for faster search on large datasets.

**Learning objectives:**
- Understand IVF (Inverted File) index
- Work with HNSW (Hierarchical Navigable Small World) index
- Compare performance vs accuracy trade-offs
- Choose the right index for your use case

In [ ]:
import faiss
import numpy as np
import time
import matplotlib.pyplot as plt

## Setup: Generate Large Dataset

Create dataset where Flat index becomes slow.

In [ ]:
# Large corpus
n_vectors = 500_000  # 500K vectors
dimension = 384

print("Generating corpus...")
corpus = np.random.randn(n_vectors, dimension).astype('float32')
faiss.normalize_L2(corpus)  # In-place normalization

print(f"Corpus: {corpus.shape}")
print(f"Memory: {corpus.nbytes / 1e9:.2f} GB")

# Test queries
n_queries = 100
queries = np.random.randn(n_queries, dimension).astype('float32')
faiss.normalize_L2(queries)

print(f"Queries: {queries.shape}")

## 1. Baseline: IndexFlatIP

Exact search - 100% accurate but slow on large datasets.

In [ ]:
# Create flat index
index_flat = faiss.IndexFlatIP(dimension)
index_flat.add(corpus)

print(f"Index size: {index_flat.ntotal:,} vectors")

In [ ]:
# Benchmark flat index
k = 10

start = time.time()
D_flat, I_flat = index_flat.search(queries, k)
time_flat = (time.time() - start) / len(queries) * 1000  # ms per query

print(f"IndexFlatIP: {time_flat:.2f} ms/query")
print(f"Total time for {len(queries)} queries: {time_flat * len(queries):.0f} ms")

## 2. IVF Index (Inverted File)

Approximate search using clustering.

**How it works:**
1. Cluster vectors into `nlist` groups
2. At search time, only check `nprobe` closest clusters
3. Much faster, slight accuracy loss

In [ ]:
# Create IVF index
nlist = 1000  # Number of clusters

# Step 1: Create quantizer (for clustering)
quantizer = faiss.IndexFlatIP(dimension)

# Step 2: Create IVF index
index_ivf = faiss.IndexIVFFlat(
    quantizer,
    dimension,
    nlist,
    faiss.METRIC_INNER_PRODUCT
)

print(f"Created IVF index with nlist={nlist}")
print(f"Is trained: {index_ivf.is_trained}")  # Should be False

In [ ]:
# Step 3: Train (learn cluster centroids)
print("Training IVF index...")
start = time.time()

# Use sample for training (faster)
train_size = min(100_000, len(corpus))
index_ivf.train(corpus[:train_size])

train_time = time.time() - start
print(f"Training took {train_time:.1f}s")
print(f"Is trained: {index_ivf.is_trained}")  # Should be True

In [ ]:
# Step 4: Add vectors
print("Adding vectors to IVF index...")
start = time.time()
index_ivf.add(corpus)
add_time = time.time() - start

print(f"Added {index_ivf.ntotal:,} vectors in {add_time:.1f}s")
print(f"Avg vectors per cluster: {index_ivf.ntotal / nlist:.0f}")

### Tuning nprobe

Control speed vs accuracy trade-off with `nprobe`.

In [ ]:
# Test different nprobe values
nprobe_values = [1, 5, 10, 20, 50]
results = []

for nprobe in nprobe_values:
    index_ivf.nprobe = nprobe

    # Measure search time
    start = time.time()
    D_ivf, I_ivf = index_ivf.search(queries, k)
    search_time = (time.time() - start) / len(queries) * 1000

    # Calculate recall@k
    recall = 0
    for i in range(len(queries)):
        true_set = set(I_flat[i])
        pred_set = set(I_ivf[i])
        recall += len(true_set & pred_set) / k
    recall /= len(queries)

    results.append({
        'nprobe': nprobe,
        'time': search_time,
        'recall': recall
    })

    print(f"nprobe={nprobe:3d}: {search_time:6.2f} ms/query, recall={recall:.4f}")

In [ ]:
# Plot results
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

# Latency vs nprobe
ax1.plot([r['nprobe'] for r in results], [r['time'] for r in results], 'o-')
ax1.axhline(time_flat, color='r', linestyle='--', label='Flat index')
ax1.set_xlabel('nprobe')
ax1.set_ylabel('Latency (ms/query)')
ax1.set_title('Search Speed vs nprobe')
ax1.legend()
ax1.grid(True)

# Recall vs nprobe
ax2.plot([r['nprobe'] for r in results], [r['recall'] for r in results], 'o-')
ax2.axhline(1.0, color='r', linestyle='--', label='Perfect recall')
ax2.set_xlabel('nprobe')
ax2.set_ylabel('Recall@10')
ax2.set_title('Recall vs nprobe')
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.show()

**Key insight:** `nprobe=10` gives ~95% recall with 5-10x speedup!

## 3. HNSW Index

Graph-based approximate search.

**Advantages:**
- Very fast (< 1ms per query)
- High recall (95-99%)
- No training needed

**Disadvantages:**
- Higher memory usage
- No deletion support

In [ ]:
# Create HNSW index
M = 8  # Number of connections per layer

index_hnsw = faiss.IndexHNSWFlat(dimension, M)
index_hnsw.hnsw.efConstruction = 64  # Build quality (set before adding)

print(f"Created HNSW with M={M}")
print(f"efConstruction={index_hnsw.hnsw.efConstruction}")

In [ ]:
# Add vectors (no training needed!)
print("Building HNSW index...")
start = time.time()
index_hnsw.add(corpus)
build_time = time.time() - start

print(f"Built index with {index_hnsw.ntotal:,} vectors in {build_time:.1f}s")

### Tuning efSearch

Control search quality with `efSearch`.

In [ ]:
# Test different efSearch values
efSearch_values = [16, 32, 64, 128, 256]
hnsw_results = []

for efSearch in efSearch_values:
    index_hnsw.hnsw.efSearch = efSearch

    # Measure search time
    start = time.time()
    D_hnsw, I_hnsw = index_hnsw.search(queries, k)
    search_time = (time.time() - start) / len(queries) * 1000

    # Calculate recall
    recall = 0
    for i in range(len(queries)):
        true_set = set(I_flat[i])
        pred_set = set(I_hnsw[i])
        recall += len(true_set & pred_set) / k
    recall /= len(queries)

    hnsw_results.append({
        'efSearch': efSearch,
        'time': search_time,
        'recall': recall
    })

    print(f"efSearch={efSearch:3d}: {search_time:6.2f} ms/query, recall={recall:.4f}")

In [ ]:
# Plot HNSW results
fig, ax = plt.subplots(figsize=(8, 5))

times = [r['time'] for r in hnsw_results]
recalls = [r['recall'] for r in hnsw_results]

ax.plot(times, recalls, 'o-', label='HNSW', linewidth=2)
ax.scatter([time_flat], [1.0], color='r', s=100, label='Flat (exact)', zorder=5)

ax.set_xlabel('Latency (ms/query)')
ax.set_ylabel('Recall@10')
ax.set_title('HNSW: Latency vs Recall Trade-off')
ax.legend()
ax.grid(True)

plt.tight_layout()
plt.show()

## 4. Index Comparison

Compare all three indexes side-by-side.

In [ ]:
# Set optimal parameters
index_ivf.nprobe = 10
index_hnsw.hnsw.efSearch = 64

# Benchmark all
indexes = [
    ('Flat', index_flat),
    ('IVF', index_ivf),
    ('HNSW', index_hnsw)
]

print(f"{'Index':<10} {'Latency (ms)':<15} {'Recall@10':<12} {'Speedup':<10}")
print("-" * 50)

for name, idx in indexes:
    # Search
    start = time.time()
    D, I = idx.search(queries, k)
    latency = (time.time() - start) / len(queries) * 1000

    # Recall
    if name == 'Flat':
        recall = 1.0
        speedup = 1.0
    else:
        recall_sum = 0
        for i in range(len(queries)):
            true_set = set(I_flat[i])
            pred_set = set(I[i])
            recall_sum += len(true_set & pred_set) / k
        recall = recall_sum / len(queries)
        speedup = time_flat / latency

    print(f"{name:<10} {latency:<15.2f} {recall:<12.4f} {speedup:<10.1f}x")

## 5. When to Use Which Index?

Decision guide based on dataset size and requirements.

In [ ]:
def recommend_index(n_vectors, memory_limited=False, ultra_low_latency=False):
    """
    Recommend FAISS index based on requirements.
    """
    if n_vectors < 10_000:
        return "IndexFlatIP - Small dataset, use exact search"

    if n_vectors < 100_000:
        if ultra_low_latency:
            return "IndexHNSWFlat - Fast and accurate"
        return "IndexFlatIP or IndexHNSWFlat"

    if n_vectors < 1_000_000:
        if memory_limited:
            return "IndexIVFFlat - Good balance"
        if ultra_low_latency:
            return "IndexHNSWFlat - Best latency"
        return "IndexIVFFlat or IndexHNSWFlat"

    if n_vectors < 10_000_000:
        if memory_limited:
            return "IndexIVFPQ - Compressed"
        return "IndexIVFFlat with GPU"

    return "IndexIVFPQ with sharding or use dedicated vector DB"

# Test recommendations
test_cases = [
    (5_000, False, False),
    (50_000, False, True),
    (500_000, False, False),
    (500_000, True, False),
    (5_000_000, False, False),
]

print("Index Recommendations:\n")
for n, mem_limited, ultra_fast in test_cases:
    rec = recommend_index(n, mem_limited, ultra_fast)
    print(f"{n:>10,} vectors, mem_limited={mem_limited}, ultra_fast={ultra_fast}")
    print(f"  → {rec}\n")

## 6. Save and Load Trained Indexes

In [ ]:
# Save IVF index (preserves training)
faiss.write_index(index_ivf, "ivf_index.faiss")
print("Saved IVF index")

# Save HNSW index
faiss.write_index(index_hnsw, "hnsw_index.faiss")
print("Saved HNSW index")

In [ ]:
# Load and verify
loaded_ivf = faiss.read_index("ivf_index.faiss")
print(f"Loaded IVF: {loaded_ivf.ntotal:,} vectors")
print(f"Is trained: {loaded_ivf.is_trained}")
print(f"nlist: {loaded_ivf.nlist}")

# Set nprobe (not saved)
loaded_ivf.nprobe = 10

# Test search
D, I = loaded_ivf.search(queries[:1], k=5)
print(f"\n✓ Loaded IVF index works")

## Summary

### Index Comparison

| Index | Speed | Memory | Accuracy | Training | Best For |
|-------|-------|--------|----------|----------|----------|
| Flat | Slow | Low | 100% | No | <10K vectors |
| IVF | Fast | Low | 95-99% | Yes | 100K-10M vectors |
| HNSW | Very Fast | High | 95-99% | No | <10M, low latency |

### Key Parameters

**IVF:**
- `nlist`: Number of clusters (√N to 4√N)
- `nprobe`: Clusters to search (nlist/10 for 95%+ recall)

**HNSW:**
- `M`: Connections per layer (16-64)
- `efConstruction`: Build quality (40-200)
- `efSearch`: Search quality (16-256)

**Next:** Learn optimization techniques and GPU acceleration!